In [ ]:
# # Fraud Detection using ML, DL, GNN with Class Balancing & Anomaly Detection

# # Required Libraries
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
# !pip install torch-geometric
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import networkx as nx
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score
# from sklearn.ensemble import IsolationForest
# from imblearn.over_sampling import SMOTE
# from torch_geometric.data import Data
# from torch_geometric.utils import from_networkx
# from torch_geometric.nn import GCNConv, GATConv, SAGEConv

# # Load Data
# data = pd.read_csv("/content/PS_20174392719_1491204439457_log.csv")
# data = data[data['isFraud'].notna()]
# data['isFraud'] = data['isFraud'].astype(int)
# data = data[data['type'].isin(['TRANSFER', 'CASH_OUT'])]
# data.drop(['step', 'type', 'nameOrig', 'nameDest', 'isFlaggedFraud'], axis=1, inplace=True)
# data.dropna(inplace=True)

# # Feature Scaling
# scaler = StandardScaler()
# features = ['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']
# data[features] = scaler.fit_transform(data[features])

# # Define X and y
# X = data.drop('isFraud', axis=1)
# y = data['isFraud']

# # Apply SMOTE
# sm = SMOTE(random_state=42)
# X_res, y_res = sm.fit_resample(X, y)
# X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, stratify=y_res)
# X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
# y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
# X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
# y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

# # DNN Models
# class FNN(nn.Module):
#     def __init__(self):
#         super(FNN, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(5, 64), nn.ReLU(),
#             nn.Linear(64, 32), nn.ReLU(),
#             nn.Linear(32, 2))
#     def forward(self, x):
#         return self.net(x)

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv = nn.Sequential(
#             nn.Conv1d(1, 32, kernel_size=2),
#             nn.ReLU(),
#             nn.Flatten())
#         self.fc = nn.Sequential(
#             nn.Linear(32 * 4, 64), nn.ReLU(),
#             nn.Linear(64, 2))
#     def forward(self, x):
#         x = x.unsqueeze(1)
#         x = self.conv(x)
#         return self.fc(x)

# class RNN(nn.Module):
#     def __init__(self):
#         super(RNN, self).__init__()
#         self.rnn = nn.RNN(input_size=5, hidden_size=64, batch_first=True)
#         self.fc = nn.Linear(64, 2)
#     def forward(self, x):
#         x = x.unsqueeze(1)
#         out, _ = self.rnn(x)
#         return self.fc(out[:, -1, :])

# def train_dnn(model, X_train, y_train, X_test, y_test):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]))
#     for _ in range(5):
#         model.train()
#         optimizer.zero_grad()
#         output = model(X_train)
#         loss = loss_fn(output, y_train)
#         loss.backward()
#         optimizer.step()
#     model.eval()
#     with torch.no_grad():
#         probs = torch.softmax(model(X_test), dim=1)[:, 1]
#         preds = (probs > 0.4).int()
#         return preds

# for name, model in {"FNN": FNN(), "CNN": CNN(), "RNN": RNN()}.items():
#     print(f"\n{name}:")
#     preds = train_dnn(model, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor)
#     print("Accuracy:", accuracy_score(y_test_tensor, preds))
#     print("F1:", f1_score(y_test_tensor, preds))
#     print("Precision:", precision_score(y_test_tensor, preds))
#     print("Recall:", recall_score(y_test_tensor, preds))
#     print("R2:", r2_score(y_test_tensor, preds))

# # GNN Models
# class GCNModel(nn.Module):
#     def __init__(self, in_channels):
#         super(GCNModel, self).__init__()
#         self.conv1 = GCNConv(in_channels, 64)
#         self.conv2 = GCNConv(64, 32)
#         self.lin = nn.Linear(32, 2)
#     def forward(self, x, edge_index, edge_attr=None):
#         x = torch.relu(self.conv1(x, edge_index))
#         x = torch.relu(self.conv2(x, edge_index))
#         return self.lin(x)

# class GATModel(nn.Module):
#     def __init__(self, in_channels):
#         super(GATModel, self).__init__()
#         self.gat1 = GATConv(in_channels, 64, heads=2)
#         self.gat2 = GATConv(128, 32, heads=1)
#         self.lin = nn.Linear(32, 2)
#     def forward(self, x, edge_index, edge_attr=None):
#         x = torch.relu(self.gat1(x, edge_index))
#         x = torch.relu(self.gat2(x, edge_index))
#         return self.lin(x)

# class SAGEModel(nn.Module):
#     def __init__(self, in_channels):
#         super(SAGEModel, self).__init__()
#         self.sage1 = SAGEConv(in_channels, 64)
#         self.sage2 = SAGEConv(64, 32)
#         self.lin = nn.Linear(32, 2)
#     def forward(self, x, edge_index, edge_attr=None):
#         x = torch.relu(self.sage1(x, edge_index))
#         x = torch.relu(self.sage2(x, edge_index))
#         return self.lin(x)

# class LSTMGCNHybrid(nn.Module):
#     def __init__(self, in_channels):
#         super().__init__()
#         self.lstm = nn.LSTM(in_channels, 64, batch_first=True)
#         self.gcn = GCNConv(64, 32)
#         self.lin = nn.Linear(32, 2)
#     def forward(self, x, edge_index, edge_attr=None):
#         x = x.unsqueeze(1)
#         x, _ = self.lstm(x)
#         x = x.squeeze(1)
#         x = torch.relu(self.gcn(x, edge_index))
#         return self.lin(x)

# # Dummy graph for demonstration
# graph = nx.DiGraph()
# for i in range(len(X)):
#     graph.add_edge(f"n{i}", f"m{i}", amount=X.iloc[i][0], isFraud=y.iloc[i])
#     graph.nodes[f"n{i}"]['x'] = [0.0]
#     graph.nodes[f"m{i}"]['x'] = [0.0]
# data_g = from_networkx(graph)
# data_g.edge_attr = torch.tensor([[e['amount']] for _, _, e in graph.edges(data=True)], dtype=torch.float)
# data_g.y = torch.tensor([e['isFraud'] for _, _, e in graph.edges(data=True)], dtype=torch.long)
# data_g.x = torch.tensor([[0.0]] * data_g.num_nodes, dtype=torch.float)

# def train_gnn(model, data):
#     optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.CrossEntropyLoss()
#     for _ in range(5):
#         model.train()
#         optimizer.zero_grad()
#         out = model(data.x, data.edge_index, data.edge_attr)
#         pred = out[data.edge_index[1]]
#         loss = loss_fn(pred, data.y)
#         loss.backward()
#         optimizer.step()
#     model.eval()
#     with torch.no_grad():
#         pred = model(data.x, data.edge_index, data.edge_attr)[data.edge_index[1]].argmax(dim=1)
#         print("Accuracy:", accuracy_score(data.y, pred))
#         print("F1:", f1_score(data.y, pred))
#         print("Precision:", precision_score(data.y, pred))
#         print("Recall:", recall_score(data.y, pred))
#         print("R2:", r2_score(data.y, pred))

# for name, model in {"GCN": GCNModel(1), "GAT": GATModel(1), "GraphSAGE": SAGEModel(1), "LSTM+GCN": LSTMGCNHybrid(1)}.items():
#     print(f"\n{name}:")
#     train_gnn(model, data_g)

# # Anomaly Detection
# iso = IsolationForest(contamination=0.001)
# y_pred_iso = iso.fit_predict(X)
# y_pred_iso = np.where(y_pred_iso == -1, 1, 0)
# print("\nIsolation Forest:")
# print("Recall:", recall_score(y, y_pred_iso))

# # Final Conclusion
# print("\nConclusion:")
# print("1. SMOTE improves fraud detection by balancing class distribution.")
# print("2. DNNs benefit from threshold tuning and weighted loss.")
# print("3. GNNs need more semantic graph structure to perform well.")
# print("4. Anomaly detection gives useful outlier insight, but low precision.")





In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric

Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.0/494.0 kB 6.4 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
import torch.nn as nn
import torch.optim as optim
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, r2_score
from sklearn.ensemble import IsolationForest
from imblearn.over_sampling import SMOTE
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv, GATConv, SAGEConv

/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_sparse/_version_cpu.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [ ]:
import pandas as pd

# Load data


# Clean column names
 # Debug: show actual column names

import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load your data
data = pd.read_csv("/content/Synthetic_Financial_datasets_log.csv")
data.columns = data.columns.str.strip()
print("Columns:", data.columns.tolist())  # See actual columns

# Ensure the target column exists and is numeric
if 'Fraud_Label' in data.columns:
    data = data[data['Fraud_Label'].notna()]
    data['Fraud_Label'] = data['Fraud_Label'].astype(int)

# Drop irrelevant or non-existent columns safely
for col in ['step', 'type', 'nameOrig', 'nameDest', 'isFlaggedFraud']:
    if col in data.columns:
        data.drop(col, axis=1, inplace=True)

# Drop any remaining missing values
data.dropna(inplace=True)



Columns: ['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']


In [ ]:
# Show exact columns
print("Actual columns in dataset:", data.columns.tolist())

# Try fuzzy matching to find similar column names
from difflib import get_close_matches

desired_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
for feat in desired_features:
    print(f"Closest match for '{feat}':", get_close_matches(feat, data.columns.tolist(), n=1))


Actual columns in dataset: ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFraud']
Closest match for 'amount': ['amount']
Closest match for 'oldbalanceOrg': ['oldbalanceOrg']
Closest match for 'newbalanceOrig': ['newbalanceOrig']
Closest match for 'oldbalanceDest': ['oldbalanceDest']
Closest match for 'newbalanceDest': ['newbalanceDest']


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv("/content/Synthetic_Financial_datasets_log.csv")
data.columns = data.columns.str.strip()  # Clean column names

# Identify the label column
label_col = None
for possible_label in ['isFraud', 'Fraud_Label']:
    if possible_label in data.columns:
        label_col = possible_label
        break

if label_col is None:
    raise ValueError("Label column 'isFraud' or 'Fraud_Label' not found.")

# Filter by type if exists
if 'type' in data.columns:
    data = data[data['type'].isin(['TRANSFER', 'CASH_OUT'])]
    data.drop(['type'], axis=1, inplace=True)

# Drop optional columns if they exist
drop_cols = ['step', 'nameOrig', 'nameDest', 'isFlaggedFraud']
data.drop([col for col in drop_cols if col in data.columns], axis=1, inplace=True)

# Clean missing values
data.dropna(inplace=True)

# Convert label to int
data[label_col] = data[label_col].astype(int)

# Normalize numeric features (only if they exist)
desired_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
available_features = [feat for feat in desired_features if feat in data.columns]

print("Scaling these features:", available_features)

if available_features:
    scaler = StandardScaler()
    data[available_features] = scaler.fit_transform(data[available_features])
else:
    print("⚠️ No features available for scaling.")

# Final dataset split
X = data.drop(label_col, axis=1)
y = data[label_col]

print("✅ Shape of X:", X.shape)
print("✅ Class distribution in y:\n", y.value_counts())


<ipython-input-6-1445930222>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['type'], axis=1, inplace=True)


Scaling these features: ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
✅ Shape of X: (838184, 5)
✅ Class distribution in y:
 isFraud
0    836294
1      1890
Name: count, dtype: int64


In [ ]:
# Keep only numeric columns in X before SMOTE
X = X.select_dtypes(include=['number'])

# Now safely apply SMOTE
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, stratify=y_res, test_size=0.2, random_state=42)

print("✅ Resampled class distribution:\n", y_res.value_counts())
print("✅ Training set size:", X_train.shape)


✅ Resampled class distribution:
 isFraud
1    836294
0    836294
Name: count, dtype: int64
✅ Training set size: (1338070, 5)


In [ ]:
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
import torch.nn as nn

class FNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.relu(self.fc1(x))
        out = self.dropout(out)
        out = self.fc2(out)
        return out


class CNNModel(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_channels, 64, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(2)
        self.dropout = nn.Dropout(0.3)
        self.fc = nn.Linear(64 * (sequence_length // 2), num_classes)  # adjust sequence_length

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)


class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # last time-step
        return out

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def evaluate(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for data, labels in dataloader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    acc = accuracy_score(all_labels, all_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

In [ ]:
def train_dnn(model, X_train, y_train, X_test, y_test):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 10.0]))
    for _ in range(5):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = loss_fn(output, y_train)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        probs = torch.softmax(model(X_test), dim=1)[:, 1]
        preds = (probs > 0.4).int()
        return preds

In [ ]:
import torch.nn as nn

class FNN(nn.Module):
    def __init__(self, input_dim=11):  # match with X_train_tensor.shape[1]
        super(FNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


In [ ]:



# Required Libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import networkx as nx
import torch
from torch_geometric.utils import from_networkx

# Step 1: Load Data
file_path = '/content/Synthetic_Financial_datasets_log.csv'  # change path if needed
data = pd.read_csv(file_path)
data.columns = data.columns.str.strip()  # Clean column names
print("🔍 Columns in dataset:", data.columns.tolist())

# Step 2: Identify Required Columns
required_cols = ['Transaction_Amount', 'Account_Balance', 'Risk_Score', 'Transaction_Distance', 'Fraud_Label']
available_cols = [col for col in required_cols if col in data.columns]
missing_cols = [col for col in required_cols if col not in data.columns]

if missing_cols:
    print(f"⚠️ Warning: Missing required columns: {missing_cols}")
if len(available_cols) < 2 or 'Fraud_Label' not in available_cols:
    raise ValueError("🚫 Not enough valid columns to continue. Please check your dataset.")

# Step 3: Drop Irrelevant Columns
drop_cols = ['Transaction_ID', 'User_ID', 'Timestamp', 'Location', 'Card_Type', 'Authentication_Method']
data.drop(columns=[col for col in drop_cols if col in data.columns], inplace=True)

# Step 4: Scale Numerical Features
features = ['Transaction_Amount', 'Account_Balance', 'Risk_Score', 'Transaction_Distance']
scaler = StandardScaler()
data[features] = scaler.fit_transform(data[features])

# Step 5: Split into X and y
label_col = 'Fraud_Label'
X = data[features].copy()
y = data[label_col].astype(int)

# Step 6: Apply SMOTE
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, y)

# Step 7: Create Graph
graph = nx.DiGraph()
for i in range(len(X_res)):
    amt = X_res.iloc[i].get('Transaction_Amount', 0.0)
    fraud = y_res.iloc[i]
    graph.add_edge(f"n{i}", f"m{i}", amount=amt, isFraud=fraud)
    graph.nodes[f"n{i}"]['x'] = [0.0]
    graph.nodes[f"m{i}"]['x'] = [0.0]

# Step 8: Convert to PyTorch Geometric Data
data_g = from_networkx(graph)
data_g.edge_attr = torch.tensor([[e['amount']] for _, _, e in graph.edges(data=True)], dtype=torch.float)
data_g.y = torch.tensor([e['isFraud'] for _, _, e in graph.edges(data=True)], dtype=torch.long)
data_g.x = torch.tensor([[0.0]] * data_g.num_nodes, dtype=torch.float)

# ✅ Final Printout
print(data_g)
print(f"📊 Number of nodes: {data_g.num_nodes}")
print(f"🔗 Number of edges: {data_g.num_edges}")
print(f"📈 Fraud class distribution: {torch.bincount(data_g.y)}")



🔍 Columns in dataset: ['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']
⚠️ Warning: Missing required columns: ['Transaction_Amount', 'Account_Balance', 'Risk_Score', 'Transaction_Distance', 'Fraud_Label']


ValueError: 🚫 Not enough valid columns to continue. Please check your dataset.

In [ ]:
def train_gnn(model, data):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    loss_fn = nn.CrossEntropyLoss()
    for _ in range(5):
        model.train()
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr)
        pred = out[data.edge_index[1]]
        loss = loss_fn(pred, data.y)
        loss.backward()
        optimizer.step()
    model.eval()
    with torch.no_grad():
        pred = model(data.x, data.edge_index, data.edge_attr)[data.edge_index[1]].argmax(dim=1)
        print("Accuracy:", accuracy_score(data.y, pred))
        print("F1:", f1_score(data.y, pred))
        print("Precision:", precision_score(data.y, pred))
        print("Recall:", recall_score(data.y, pred))
        print("R2:", r2_score(data.y, pred))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv, SAGEConv
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

# Load dataset
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
data_g = dataset[0]  # Cora dataset has only one graph
input_dim = dataset.num_node_features
num_classes = dataset.num_classes  # Cora has 7 classes

# GCN Model
class GCNModel(nn.Module):
    def __init__(self, input_dim):
        super(GCNModel, self).__init__()
        self.conv1 = GCNConv(input_dim, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

# GAT Model
class GATModel(nn.Module):
    def __init__(self, input_dim):
        super(GATModel, self).__init__()
        self.gat1 = GATConv(input_dim, 8, heads=4, concat=True)
        self.gat2 = GATConv(32, num_classes, heads=1, concat=False)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        return x

# GraphSAGE Model
from torch_geometric.nn import SAGEConv

class ImprovedGraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(ImprovedGraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.bn1 = nn.BatchNorm1d(hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x



# LSTM + GCN Hybrid Model
class LSTMGCN(torch.nn.Module):
    def __init__(self, input_size, hidden_size, gcn_in, gcn_out, num_classes):
        super(LSTMGCN, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.gcn = GCNConv(gcn_in, gcn_out)
        self.fc = nn.Linear(hidden_size + gcn_out, num_classes)

    def forward(self, x_seq, x_graph, edge_index):
        lstm_out, _ = self.lstm(x_seq)
        lstm_feat = lstm_out[:, -1, :]  # last timestep

        gcn_feat = self.gcn(x_graph, edge_index)

        combined = torch.cat([lstm_feat, gcn_feat], dim=1)
        out = self.fc(combined)
        return out

# Training Function
def train_gnn(model, data):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    data = data.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(20):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1} - Loss: {loss.item():.4f}")

# Main Loop for Training and Evaluation

  # Main Loop for Training and Evaluation

models = {
    "GCNModel": GCNModel(input_dim),
    "GAT": GATModel(input_dim),
    "GraphSAGE": ImprovedGraphSAGE(input_dim, 16, num_classes),
    "LSTM+GCN": LSTMGCN(input_size=input_dim, hidden_size=16, gcn_in=input_dim, gcn_out=16, num_classes=num_classes)
}

for name, model in models.items():
    print(f"\n{name}:")
    train_gnn(model, data_g)

    model.eval()
    with torch.no_grad():
        out = model(data_g)
        y_pred = out.argmax(dim=1)
        y_true = data_g.y

        y_pred_test = y_pred[data_g.test_mask].cpu()
        y_true_test = y_true[data_g.test_mask].cpu()

        print("Confusion Matrix:")
        print(confusion_matrix(y_true_test, y_pred_test))

        print("Classification Report:")
        print(classification_report(y_true_test, y_pred_test, digits=4))

        try:
            print("AUC-ROC Score:")
            print(roc_auc_score(y_true_test, y_pred_test, multi_class='ovr'))
        except ValueError:
            print("⚠️ AUC-ROC not defined (only one class in predictions).")







In [ ]:
from sklearn.preprocessing import label_binarize

model.eval()
with torch.no_grad():
    out = model(data_g)
    y_scores = F.softmax(out, dim=1)  # Get probability distribution
    y_pred = y_scores.argmax(dim=1)
    y_true = data_g.y

    y_pred_test = y_pred[data_g.test_mask].cpu()
    y_true_test = y_true[data_g.test_mask].cpu()
    y_scores_test = y_scores[data_g.test_mask].cpu()

    print("Confusion Matrix:")
    print(confusion_matrix(y_true_test, y_pred_test))

    print("Classification Report:")
    print(classification_report(y_true_test, y_pred_test, digits=4, zero_division=0))

    try:
        # Binarize labels for ROC AUC
        y_true_bin = label_binarize(y_true_test, classes=list(range(num_classes)))
        print("AUC-ROC Score:")
        print(roc_auc_score(y_true_bin, y_scores_test, multi_class='ovr'))
    except ValueError as e:
        print(f"⚠️ AUC-ROC not defined: {e}")



In [ ]:
iso = IsolationForest(contamination=0.001)
y_pred_iso = iso.fit_predict(X)
y_pred_iso = np.where(y_pred_iso == -1, 1, 0)
print("\nIsolation Forest:")
print("Recall:", recall_score(y, y_pred_iso))

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report

def evaluate_model(preds, labels, probs=None):
    print("Confusion Matrix:\n", confusion_matrix(labels, preds))
    print("Classification Report:\n", classification_report(labels, preds, digits=4))
    if probs is not None:
        try:
            print("AUC-ROC Score:", roc_auc_score(labels, probs))
        except:
            print("⚠️ AUC undefined.")


In [ ]:
for i, row in X_res.iterrows():
    user = row['User_ID'] if 'User_ID' in row else f"user_{i % 10}"
    merchant = row['Merchant_ID'] if 'Merchant_ID' in row else f"merchant_{i % 5}"
    graph.add_edge(user, merchant, amount=row['Transaction_Amount'], isFraud=int(y_res.iloc[i]))


In [ ]:
results = []

def record_results(name, y_true, y_pred):
    results.append({
        "Model": name,
        "Accuracy": accuracy_score(y_true, y_pred),
        "F1": f1_score(y_true, y_pred),
        "Precision": precision_score(y_true, y_pred),
        "Recall": recall_score(y_true, y_pred)
    })

# After evaluation:
pd.DataFrame(results)


In [ ]:
print("\nConclusion:")
print("1. SMOTE improves fraud detection by balancing class distribution.")
print("2. DNNs benefit from threshold tuning and weighted loss.")
print("3. GNNs need more semantic graph structure to perform well.")
print("4. Anomaly detection gives useful outlier insight, but low precision.")